# 라이브러리 임포트 

In [1]:
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)
from kubernetes import client 

# KServe
- 모델 생성하기 

In [3]:
namespace = utils.get_default_target_namespace()
service_name="sklearn-iris-python"

In [4]:
api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'
image = "kserve/sklearnserver:v0.9.0"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       sklearn=(V1beta1SKLearnSpec(
                                           image = image,
                                            storage_uri = storage))))
)

In [5]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-04-28T01:01:01Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-04-28T01:00:59Z'}],
  'name': 'sklearn-iris-python',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '271722',
  'uid': '161f52d6-d3a7-4ddf-8079-39c560be581a'},
 'spec': {'predictor': {'model': {'image': 'kserve/sklearnserver:v0.9.0',
    'modelFormat': {'name': 'sklearn'},
    'name': '',
    'protocolVersion': 'v1',
    'resources': {

In [6]:
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

In [7]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-04-28T01:01:01Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-04-28T01:00:59Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {'.': {},
       'v:"inferenceservice.finalizers"': {}}},
     'f:status': {'.': {},
      'f:address': {'.': {}, 'f:url': {}},
      'f:components': {'.': {},
  

# 생성한 모델을 이용하여 예측하기 

In [8]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [9]:
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

response = requests.post(isvc_url, json=sklearn_iris_input)
print(response.text)

http://sklearn-iris-python.kubeflow-user-example-com.svc.cluster.local/v1/models/sklearn-iris-python:predict
{"predictions": [1, 1]}
